In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('cd-elim.csv')
df.head()

,Color,Toughness,Fungus,Apperance,Poisonous
0,G,H,N,W,Yes
1,G,H,Y,S,No
2,B,S,N,W,No
3,O,H,N,W,Yes
4,G,S,Y,S,Yes


In [3]:
class CandidateElimination():
    def g_0(self, n):
        return ("any",)*n

    def s_0(self, n):
        return ('0',)*n  
    
    def more_general(self, h1, h2):
        more_general_parts = []
        for x, y in zip(h1, h2):
            mg = x == "any" or (x != "0" and (x == y or y == "0"))
            more_general_parts.append(mg)
        return all(more_general_parts)
    
    def fulfills(self, example, hypothesis):
        return self.more_general(hypothesis, example)

    def min_generalizations(self, h, x):
        h_new = list(h)
        for i in range(len(h)):
            if not fulfills(x[i:i+1], h[i:i+1]):
                h_new[i] = 'any' if h[i] != '0' else x[i]
        return [tuple(h_new)]
    
    def min_specializations(self, h, domains, x):
        results = []
        for i in range(len(h)):
            if h[i] == "any":
                for val in domains[i]:
                    if x[i] != val:
                        h_new = h[:i] + (val,) + h[i+1:]
                        results.append(h_new)
            elif h[i] != "0":
                h_new = h[:i] + ('0',) + h[i+1:]
                results.append(h_new)
        return results
    
    def get_domains(self, examples):
        d = [set() for i in examples[0]]
        for x in examples:
            for i, xi in enumerate(x):
                d[i].add(xi)
        return [list(sorted(x)) for x in d]
    
    def train(self, examples):
        domains = self.get_domains(examples)[:-1]
    
        G = set([self.g_0(len(domains))])
        S = set([self.s_0(len(domains))])
        i=0
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
        for xcx in examples:
            i=i+1
            x, cx = xcx[:-1], xcx[-1] 
            if cx=='Y': 
                G = {g for g in G if self.fulfills(x, g)}
                S = self.generalize_S(x, G, S)
            else:
                S = {s for s in S if not self.fulfills(x, s)}
                G = self.specialize_G(x, domains, G, S)
            print("\n G[{0}]:".format(i),G)
            print("\n S[{0}]:".format(i),S)
        return 
    
    def generalize_S(self, x, G, S):
        S_prev = list(S)
        for s in S_prev:
            if s not in S:
                continue
            if not self.fulfills(x, s):
                S.remove(s)
                Splus = self.min_generalizations(s, x)
                S.update([h for h in Splus if any([self.more_general(g,h) for g in G])])
                S.difference_update([h for h in S if any([self.more_general(h, h1) for h1 in S if h != h1])])
        return S
    
    def specialize_G(self, x, domains, G, S):
        G_prev = list(G)
        for g in G_prev:
            if g not in G:
                continue
            if self.fulfills(x, g):
                G.remove(g)
                Gminus = self.min_specializations(g, domains, x)
                G.update([h for h in Gminus if any([self.more_general(h, s) for s in S])])
                G.difference_update([h for h in G if any([self.more_general(g1, h) for g1 in G if h != g1])])
        return G

In [4]:
dataList=[]

for row in df.iterrows():
    index, data = row
    dataList.append(data.tolist())
    
dataList

[['G', 'H', 'N', 'W', 'Yes'],
 ['G', 'H', 'Y', 'S', 'No'],
 ['B', 'S', 'N', 'W', 'No'],
 ['O', 'H', 'N', 'W', 'Yes'],
 ['G', 'S', 'Y', 'S', 'Yes'],
 ['G', 'H', 'Y', 'W', 'Yes'],
 ['O', 'H', 'N', 'W', 'Yes']]

In [5]:
model = CandidateElimination()

In [6]:
model.train(dataList)


 G[0]: {('any', 'any', 'any', 'any')}

 S[0]: {('0', '0', '0', '0')}

 G[1]: {('O', 'any', 'any', 'any'), ('any', 'any', 'Y', 'any'), ('B', 'any', 'any', 'any'), ('any', 'S', 'any', 'any'), ('any', 'any', 'any', 'S')}

 S[1]: {('0', '0', '0', '0')}

 G[2]: {('O', 'any', 'any', 'any'), ('any', 'any', 'Y', 'W'), ('B', 'any', 'any', 'any'), ('any', 'any', 'N', 'S'), ('any', 'S', 'any', 'any')}

 S[2]: {('0', '0', '0', '0')}

 G[3]: {('B', 'any', 'any', 'S'), ('G', 'S', 'any', 'any'), ('any', 'S', 'any', 'S'), ('B', 'H', 'any', 'any'), ('O', 'any', 'any', 'any'), ('any', 'any', 'Y', 'W'), ('any', 'any', 'N', 'S'), ('any', 'S', 'Y', 'any'), ('B', 'any', 'Y', 'any')}

 S[3]: {('0', '0', '0', '0')}

 G[4]: {('B', 'any', 'any', 'S'), ('G', 'S', 'any', 'any'), ('any', 'S', 'any', 'S'), ('B', 'H', 'any', 'any'), ('any', 'any', 'Y', 'W'), ('any', 'any', 'N', 'S'), ('O', 'S', 'any', 'any'), ('O', 'any', 'Y', 'any'), ('any', 'S', 'Y', 'any'), ('O', 'any', 'any', 'S'), ('B', 'any', 'Y', 'any')}

 S